In [33]:
import sys
import os
from pathlib import Path

# Handling the scenario where __file__ might not be defined
if '__file__' in globals():
    # If __file__ is defined, use it to get the current directory
    current_dir = Path(os.path.dirname(os.path.abspath(__file__)))
else:
    # If __file__ is not defined, fall back to the current working directory
    current_dir = Path(os.getcwd())

# Calculate the parent directory
parent_dir = current_dir.parent

# Append the parent directory to sys.path
sys.path.append(str(parent_dir))


In [34]:
import geopandas as gpd
import requests
import os
from shapely.geometry import Polygon, LineString, MultiLineString
import io
import osmnx as ox
from shapely import Point
from mapclassify import greedy
import momepy
import shapely
from shapely.ops import split, snap, unary_union


In [35]:
local_crs = 27700
place = "test"
lat = 55.86421405612109
lng = -4.251846930489373
country = "UK"
crs=4326
radius=1

In [36]:
latlng = (lat, lng)

In [37]:
if not os.path.exists(f'../output/{place}'):
    os.makedirs(f'../output/{place}')

## part 1: Find limits of cells to download

In [38]:
# Base URL for the FeatureServer
base_url = "https://services.arcgis.com/qHLhLQrcvEnxjtPr/ArcGIS/rest/services/OS_OpenMap_Local_Buildings/FeatureServer/1/query"

# Parameters
params = {
    "where": "",
    "objectIds": "",
    "time": "",
    "geometry": f'{{"x": {lng}, "y": {lat}, "spatialReference": {{"wkid": 4326}}}}',
    "geometryType": "esriGeometryPoint",
    "inSR": "",
    "spatialRel": "esriSpatialRelIntersects",
    "resultType": "none",
    "distance": radius+5,
    "units": "esriSRUnit_Kilometer",
    "relationParam": "",
    "returnGeodetic": "false",
    "outFields": "",
    "returnGeometry": "true",
    "returnCentroid": "false",
    "returnEnvelope": "false",
    "featureEncoding": "esriDefault",
    "multipatchOption": "xyFootprint",
    "maxAllowableOffset": "",
    "geometryPrecision": "",
    "outSR": "",
    "defaultSR": "",
    "datumTransformation": "",
    "applyVCSProjection": "false",
    "returnIdsOnly": "false",
    "returnUniqueIdsOnly": "false",
    "returnCountOnly": "false",
    "returnExtentOnly": "false",
    "returnQueryGeometry": "false",
    "returnDistinctValues": "false",
    "cacheHint": "false",
    "orderByFields": "",
    "groupByFieldsForStatistics": "",
    "outStatistics": "",
    "having": "",
    "resultOffset": "",
    "resultRecordCount": "",
    "returnZ": "false",
    "returnM": "false",
    "returnExceededLimitFeatures": "true",
    "quantizationParameters": "",
    "sqlFormat": "none",
    "f": "GEOJSON",
    "token": ""  # Add token if necessary
}

# Make the GET request
response = requests.get(base_url, params=params)

# Check the status code and print the result
if response.status_code == 200:
    print("Success:")
    print(response.text)  # Or process the response as you need
else:
    print("Failed to retrieve data")
    print(response.status_code)

Success:
{"type":"FeatureCollection","properties":{"exceededTransferLimit":true},"features":[{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-4.25313306910068,55.862694109504],[-4.25266865354838,55.8639388247231],[-4.25274204573954,55.8639477259542],[-4.25270747140481,55.8640378730892],[-4.25265248342185,55.8640542486323],[-4.25263017874684,55.8640637343815],[-4.25261532036104,55.8640720951318],[-4.25259818311673,55.8640852609062],[-4.2525855732097,55.8640993323618],[-4.25257717134645,55.8641143153482],[-4.25257579839679,55.8641182050479],[-4.25257054451802,55.8641737531158],[-4.25257259998902,55.864184410393],[-4.25258519614991,55.864211501171],[-4.25273203009428,55.8642522204834],[-4.2528469776111,55.8642501149118],[-4.25287727734242,55.8642351801415],[-4.2528973038396,55.8642222310213],[-4.25291174260348,55.8642093842578],[-4.25292223386191,55.8641918465199],[-4.25311460042626,55.8642157338155],[-4.25309128401375,55.8642767355991],[-4.25391703522479,55.8643754744721]

In [39]:
# Convert the string to a file-like object
geojson_file_like = io.StringIO(response.text)

# Read into a GeoDataFrame
downloaded_gdf = gpd.read_file(geojson_file_like)

In [40]:
downloaded_gdf.explore()

In [41]:
buffer = gpd.GeoDataFrame(index=[0], geometry=[Point(lng, lat)], crs=4326).to_crs(local_crs).buffer((radius+5)*1000).to_crs(4326)

/home/virgilxw/miniforge3/envs/downloader/lib/python3.11/site-packages/pyproj/transformer.py:820: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return self._transformer._transform_point(


In [42]:
buffer.explore()

In [43]:
water_gdf = ox.geometries_from_polygon(buffer[0], tags={"natural": "water"})
water_gdf = water_gdf[water_gdf.geometry.type.isin(['Polygon', 'MultiPolygon'])]

/tmp/ipykernel_32213/2221155826.py:1: UserWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in a future release.
  water_gdf = ox.geometries_from_polygon(buffer[0], tags={"natural": "water"})


In [44]:
# List all columns to drop, which is all except 'geometry'
columns_to_drop = [col for col in water_gdf.columns if col != 'geometry']

# Drop these columns
water_gdf = water_gdf.drop(columns=columns_to_drop)

water_gdf = water_gdf.reset_index(drop=True)

In [45]:
water_gdf.to_parquet(f"../output/{place}_OS/water.pq")

In [46]:
buffer = gpd.GeoDataFrame(geometry=buffer, crs=4326)

In [47]:
if country == None:
    world_poly = gpd.read_file(f"../input/{place}_study_area.shp")
else:
    world_poly = gpd.read_file(f"../input/{country}_study_area.shp")

In [48]:
study_area = gpd.overlay(world_poly, buffer, how='intersection').overlay(water_gdf, how='difference')

In [49]:
streets = gpd.read_parquet("../input/gitignore/os-roads/merged_roads.parquet")

In [50]:
streets = streets.to_crs(4326)

In [ ]:
# List of all column names to be dropped
columns_to_drop = [col for col in streets.columns if col != 'geometry']

# Drop the columns
streets = streets.drop(columns=columns_to_drop)

In [ ]:
streets = gpd.sjoin(streets, buffer, how="inner", op="intersects")

In [ ]:
# streets.explore()

In [ ]:
streets = streets.drop(columns="index_right")

In [ ]:
streets = streets.explode()

In [ ]:

# Assuming you have a GeoDataFrame named 'gdf' with your data

def convert_geometry(geom):
    if geom.geom_type == 'LineStringZ':
        return LineString([(x, y) for x, y, z in geom.coords])
    else:
        return geom  # Return the geometry unchanged if it's not LineStringZ or MultiLineString

# Apply the conversion to each geometry in the GeoDataFrame
streets['geometry'] = streets['geometry'].apply(convert_geometry)

In [ ]:
downloaded_gdf.to_parquet(f"../output/{place}_OS/buildings_raw.pq")

In [ ]:
streets.to_parquet(f"../output/{place}_OS/streets_raw.pq")

In [ ]:
osm_graph= ox.graph.graph_from_polygon(buffer.geometry[0], network_type='all', custom_filter='["railway"~"rail"]["tunnel"!="yes"]')
osm_graph = ox.projection.project_graph(osm_graph, to_crs=local_crs)
rail = ox.graph_to_gdfs(
    ox.get_undirected(osm_graph),
    nodes=False,
    edges=True,
    node_geometry=False,
    fill_edge_geometry=True
    
)

rail.head()

# List all columns to drop, which is all except 'geometry'
columns_to_drop = [col for col in rail.columns if col != 'geometry']

# Drop these columns
rail = rail.drop(columns=columns_to_drop).reset_index(drop=True)

# Assuming your GeoDataFrame is named 'gdf'
rail_strings = []
for geometry in rail.geometry:
    if geometry.geom_type == 'LineString':
        rail_strings.append(geometry)
    elif geometry.geom_type == 'MultiLineString':
        rail_strings.extend(list(geometry))
        
collection = shapely.GeometryCollection(rail_strings)  # combine to a single object
noded = shapely.node(collection)  # add missing nodes
rail_noded_gdf = gpd.GeoDataFrame(geometry=gpd.GeoSeries(noded), crs=local_crs)

In [ ]:
rail_noded_gdf.explore()

In [ ]:
rail_noded_gdf.to_parquet(f"../output/{place}_OS/rail_raw.pq")

In [ ]:
study_area.to_parquet((f"../output/{place}_OS/study_area.pq"))

In [ ]:
study_area.plot()